In [71]:
import numpy as np
import pandas as pd
import re

In [72]:
raw_data = pd.read_csv("../data/raw/diabetic_data.csv")

In [73]:
missing_value_count = (raw_data == "?").sum()

missing_value_percentage = ((raw_data == "?").sum() / len(raw_data)) * 100

missing_value_percentage

encounter_id                 0.000000
patient_nbr                  0.000000
race                         2.233555
gender                       0.000000
age                          0.000000
weight                      96.858479
admission_type_id            0.000000
discharge_disposition_id     0.000000
admission_source_id          0.000000
time_in_hospital             0.000000
payer_code                  39.557416
medical_specialty           49.082208
num_lab_procedures           0.000000
num_procedures               0.000000
num_medications              0.000000
number_outpatient            0.000000
number_emergency             0.000000
number_inpatient             0.000000
diag_1                       0.020636
diag_2                       0.351787
diag_3                       1.398306
number_diagnoses             0.000000
max_glu_serum                0.000000
A1Cresult                    0.000000
metformin                    0.000000
repaglinide                  0.000000
nateglinide 

### Drop columns that exceed the 20% acceptable threshold for missing values.

In [74]:
raw_data = raw_data.drop(columns=["weight", "payer_code", "medical_specialty"], inplace=False)

raw_data.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,41,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [75]:
DRUG_COLS = [
    "metformin","repaglinide","nateglinide","chlorpropamide","glimepiride",
    "acetohexamide","glipizide","glyburide","tolbutamide","pioglitazone",
    "rosiglitazone","acarbose","miglitol","troglitazone","tolazamide",
    "examide","citoglipton","insulin","glyburide-metformin","glipizide-metformin",
    "glimepiride-pioglitazone","metformin-rosiglitazone","metformin-pioglitazone"
]

drug_summary = raw_data[DRUG_COLS].apply(pd.Series.value_counts).T.fillna(0).astype(int)
drug_percentages = drug_summary.div(drug_summary.sum(axis=1), axis=0) * 100
  
drug_percentages

,Down,No,Steady,Up
metformin,0.565022,80.358862,18.027632,1.048484
repaglinide,0.044219,98.487707,1.359983,0.108091
nateglinide,0.010809,99.309200,0.656408,0.023584
chlorpropamide,0.000983,99.915492,0.077629,0.005896
glimepiride,0.190633,94.899082,4.588959,0.321325
acetohexamide,0.000000,99.999017,0.000983,0.000000
glipizide,0.550282,87.534147,11.158933,0.756638
glyburide,0.554213,89.534815,9.113063,0.797909
tolbutamide,0.000000,99.977399,0.022601,0.000000
pioglitazone,0.115952,92.799167,6.854942,0.229939


### Drop columns that are near constant (>= 99%).

In [76]:
drop_drugs = [
    "nateglinide",
    "chlorpropamide",
    "acetohexamide",
    "tolbutamide",
    "acarbose",
    "miglitol",
    "troglitazone",
    "tolazamide",
    "examide",
    "citoglipton",
    "glyburide-metformin",
    "glipizide-metformin",
    "glimepiride-pioglitazone",
    "metformin-rosiglitazone",
    "metformin-pioglitazone"
]

raw_data = raw_data.drop(columns=drop_drugs, inplace=False)
raw_data.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,repaglinide,glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,insulin,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,41,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,...,No,No,No,No,No,No,Up,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,...,No,No,Steady,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,...,No,No,No,No,No,No,Up,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,No,No,Steady,No,No,No,Steady,Ch,Yes,NO


In [77]:
patient_nbr_is_unique = raw_data["patient_nbr"].is_unique

encounter_id_is_unique = raw_data["encounter_id"].is_unique

print(f"Patient number is unique: {patient_nbr_is_unique}")
print(f"Encounter id is unique: {encounter_id_is_unique}")

Patient number is unique: False
Encounter id is unique: True


In [78]:
raw_data = raw_data.drop(columns=["patient_nbr", "encounter_id"], inplace=False)

raw_data.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,repaglinide,glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,insulin,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,25,1,1,41,0,1,...,No,No,No,No,No,No,No,No,No,NO
1,Caucasian,Female,[10-20),1,1,7,3,59,0,18,...,No,No,No,No,No,No,Up,Ch,Yes,>30
2,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,...,No,No,Steady,No,No,No,No,No,Yes,NO
3,Caucasian,Male,[30-40),1,1,7,2,44,1,16,...,No,No,No,No,No,No,Up,Ch,Yes,NO
4,Caucasian,Male,[40-50),1,1,7,1,51,0,8,...,No,No,Steady,No,No,No,Steady,Ch,Yes,NO


In [79]:
OHE_COLS = [
    "race",
    "gender",
    "admission_type_id",
    "discharge_disposition_id",
    "admission_source_id",
    "max_glu_serum",
    "A1Cresult"
]

raw_data = pd.get_dummies(raw_data, columns=OHE_COLS, drop_first=False, dtype=int)

raw_data.head()

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,...,admission_source_id_17,admission_source_id_20,admission_source_id_22,admission_source_id_25,max_glu_serum_>200,max_glu_serum_>300,max_glu_serum_Norm,A1Cresult_>7,A1Cresult_>8,A1Cresult_Norm
0,[0-10),1,41,0,1,0,0,0,250.83,?,...,0,0,0,0,0,0,0,0,0,0
1,[10-20),3,59,0,18,0,0,0,276,250.01,...,0,0,0,0,0,0,0,0,0,0
2,[20-30),2,11,5,13,2,0,1,648,250,...,0,0,0,0,0,0,0,0,0,0
3,[30-40),2,44,1,16,0,0,0,8,250.43,...,0,0,0,0,0,0,0,0,0,0
4,[40-50),1,51,0,8,0,0,0,197,157,...,0,0,0,0,0,0,0,0,0,0


In [80]:
age_map = {
    "[0-10)":5, "[10-20)":15, "[20-30)":25, "[30-40)":35,
    "[40-50)":45, "[50-60)":55, "[60-70)":65, "[70-80)":75,
    "[80-90)":85, "[90-100)":95
}
raw_data["age"] = raw_data["age"].map(age_map).astype(int)
raw_data.head()

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,...,admission_source_id_17,admission_source_id_20,admission_source_id_22,admission_source_id_25,max_glu_serum_>200,max_glu_serum_>300,max_glu_serum_Norm,A1Cresult_>7,A1Cresult_>8,A1Cresult_Norm
0,5,1,41,0,1,0,0,0,250.83,?,...,0,0,0,0,0,0,0,0,0,0
1,15,3,59,0,18,0,0,0,276,250.01,...,0,0,0,0,0,0,0,0,0,0
2,25,2,11,5,13,2,0,1,648,250,...,0,0,0,0,0,0,0,0,0,0
3,35,2,44,1,16,0,0,0,8,250.43,...,0,0,0,0,0,0,0,0,0,0
4,45,1,51,0,8,0,0,0,197,157,...,0,0,0,0,0,0,0,0,0,0


In [81]:
# 

DRUG_KEEP = [
    "metformin","repaglinide","glimepiride","glipizide","glyburide",
    "pioglitazone","rosiglitazone","insulin"
]
drug_map = {"No":0, "Down":1, "Steady":2, "Up":3}
for col in DRUG_KEEP:
    raw_data[col] = raw_data[col].map(drug_map).astype(int)

raw_data.head()

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,...,admission_source_id_17,admission_source_id_20,admission_source_id_22,admission_source_id_25,max_glu_serum_>200,max_glu_serum_>300,max_glu_serum_Norm,A1Cresult_>7,A1Cresult_>8,A1Cresult_Norm
0,5,1,41,0,1,0,0,0,250.83,?,...,0,0,0,0,0,0,0,0,0,0
1,15,3,59,0,18,0,0,0,276,250.01,...,0,0,0,0,0,0,0,0,0,0
2,25,2,11,5,13,2,0,1,648,250,...,0,0,0,0,0,0,0,0,0,0
3,35,2,44,1,16,0,0,0,8,250.43,...,0,0,0,0,0,0,0,0,0,0
4,45,1,51,0,8,0,0,0,197,157,...,0,0,0,0,0,0,0,0,0,0


In [82]:
binary_map = {"No":0, "Ch":1, "Yes":1}
raw_data["change"] = raw_data["change"].map(binary_map).astype(int)
raw_data["diabetesMed"] = raw_data["diabetesMed"].map(binary_map).astype(int)

raw_data.head()

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,...,admission_source_id_17,admission_source_id_20,admission_source_id_22,admission_source_id_25,max_glu_serum_>200,max_glu_serum_>300,max_glu_serum_Norm,A1Cresult_>7,A1Cresult_>8,A1Cresult_Norm
0,5,1,41,0,1,0,0,0,250.83,?,...,0,0,0,0,0,0,0,0,0,0
1,15,3,59,0,18,0,0,0,276,250.01,...,0,0,0,0,0,0,0,0,0,0
2,25,2,11,5,13,2,0,1,648,250,...,0,0,0,0,0,0,0,0,0,0
3,35,2,44,1,16,0,0,0,8,250.43,...,0,0,0,0,0,0,0,0,0,0
4,45,1,51,0,8,0,0,0,197,157,...,0,0,0,0,0,0,0,0,0,0


In [83]:
# Bucket ICD-9 codes for interpretability
# Example of common buckets:
# - Cardiovascular Disease
# - Kidney Disease
# - Respiratory Disease

for c in ["diag_1", "diag_2", "diag_3"]:
    raw_data[c] = raw_data[c].replace("?", "UNK").astype(str)

def bucket_icd9(code: str) -> str:
    """
    Returns a coarse clinical group for an ICD-9 diagnosis code.
    - Handles UNK / missing
    - Handles 'V' and 'E' codes
    - Uses the first 3 digits for numeric codes (e.g., 250.13 -> 250)
    """
    if not code or code.upper() in {"UNK", "UNKNOWN", "NA", "NONE"}:
        return "Unknown"

    code = code.strip()

    # V and E codes (non-numeric ICD-9 chapters)
    if code[0] in ("V", "v"):
        return "Supplementary Factors (V)"
    if code[0] in ("E", "e"):
        return "External Causes (E)"

    # Extract first 3 digits from numeric codes (handles decimals like 250.13)
    m = re.match(r"^(\d{3})", code.replace(".", ""))
    if not m:
        return "Unknown"

    three = int(m.group(1))

    # --- Buckets commonly used with this dataset ---
    # Infectious & Parasitic
    if   1 <= three <= 139:   return "Infectious & Parasitic (001–139)"
    # Neoplasms
    if 140 <= three <= 239:   return "Neoplasms (140–239)"
    # Endocrine, Nutritional, Metabolic (includes diabetes 250.*)
    if 240 <= three <= 279:
        if three == 250:      return "Diabetes (250)"
        return "Endocrine/Metabolic (240–279)"
    # Diseases of the Blood
    if 280 <= three <= 289:   return "Blood (280–289)"
    # Mental Disorders
    if 290 <= three <= 319:   return "Mental (290–319)"
    # Nervous System & Sense Organs
    if 320 <= three <= 389:   return "Nervous/Sense (320–389)"
    # Circulatory System
    if 390 <= three <= 459:   return "Circulatory (390–459)"
    # Respiratory System
    if 460 <= three <= 519:   return "Respiratory (460–519)"
    # Digestive System
    if 520 <= three <= 579:   return "Digestive (520–579)"
    # Genitourinary System
    if 580 <= three <= 629:   return "Genitourinary (580–629)"
    # Pregnancy, Childbirth, Puerperium
    if 630 <= three <= 679:   return "Pregnancy (630–679)"
    # Skin & Subcutaneous Tissue
    if 680 <= three <= 709:   return "Skin (680–709)"
    # Musculoskeletal & Connective Tissue
    if 710 <= three <= 739:   return "Musculoskeletal (710–739)"
    # Congenital Anomalies
    if 740 <= three <= 759:   return "Congenital (740–759)"
    # Certain Conditions Originating in Perinatal Period
    if 760 <= three <= 779:   return "Perinatal (760–779)"
    # Symptoms, Signs, & Ill-defined Conditions
    if 780 <= three <= 799:   return "Symptoms/Ill-defined (780–799)"
    # Injury & Poisoning
    if 800 <= three <= 999:   return "Injury/Poisoning (800–999)"

    return "Other/Unmapped"

# 3) Apply to diag_1..3 to create grouped columns
for c in ["diag_1", "diag_2", "diag_3"]:
    raw_data[c + "_grp"] = raw_data[c].apply(bucket_icd9)

# (Optional) Drop raw ICD columns if you won’t use them directly
# raw_data = raw_data.drop(columns=["diag_1","diag_2","diag_3"])

# 4) (Optional) One-hot encode the new groups
DIAG_GRP_COLS = ["diag_1_grp", "diag_2_grp", "diag_3_grp"]
raw_data = pd.get_dummies(raw_data, columns=DIAG_GRP_COLS, drop_first=False, dtype=int)

raw_data.head()

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,...,diag_3_grp_Mental (290–319),diag_3_grp_Musculoskeletal (710–739),diag_3_grp_Neoplasms (140–239),diag_3_grp_Nervous/Sense (320–389),diag_3_grp_Pregnancy (630–679),diag_3_grp_Respiratory (460–519),diag_3_grp_Skin (680–709),diag_3_grp_Supplementary Factors (V),diag_3_grp_Symptoms/Ill-defined (780–799),diag_3_grp_Unknown
0,5,1,41,0,1,0,0,0,250.83,UNK,...,0,0,0,0,0,0,0,0,0,1
1,15,3,59,0,18,0,0,0,276,250.01,...,0,0,0,0,0,0,0,0,0,0
2,25,2,11,5,13,2,0,1,648,250,...,0,0,0,0,0,0,0,1,0,0
3,35,2,44,1,16,0,0,0,8,250.43,...,0,0,0,0,0,0,0,0,0,0
4,45,1,51,0,8,0,0,0,197,157,...,0,0,0,0,0,0,0,0,0,0


In [85]:
# --- 2) drop raw ICD code columns (now redundant after *_grp OHE) ---
raw_data = raw_data.drop(columns=["diag_1", "diag_2", "diag_3"], errors="ignore")

# --- 3) add medication aggregates (from the drugs you kept) ---
DRUG_KEEP = [
    "metformin","repaglinide","glimepiride","glipizide","glyburide",
    "pioglitazone","rosiglitazone","insulin"
]
# (They are already ordinal-encoded: No=0, Down=1, Steady=2, Up=3)
raw_data["num_diabetes_meds_used"] = raw_data[DRUG_KEEP].apply(lambda s: s.isin([1,2,3]).sum(), axis=1)
raw_data["num_med_dose_changes"]  = raw_data[DRUG_KEEP].apply(lambda s: s.isin([1,3]).sum(), axis=1)

# --- 4) (optional) replace any lingering "?" with NaN then fill or drop as you prefer ---
raw_data = raw_data.replace("?", np.nan)

# Example: if any remaining NaNs are in OHE’d columns (rare), fill with 0
# raw_data = raw_data.fillna(0)

# --- 5) sanity checks before modeling ---
# a) confirm all features except the target are numeric
non_numeric = raw_data.drop(columns=["readmitted"], errors="ignore").select_dtypes(include=["object"]).columns.tolist()
if non_numeric:
    print("Non-numeric columns remain:", non_numeric)
else:
    print("All feature columns are numeric.")

# b) quick skew check (optional): consider log1p transforms later for these
skew_candidates = ["number_outpatient","number_emergency","number_inpatient"]
print("Skew candidates present:", [c for c in skew_candidates if c in raw_data.columns])

raw_data.head()


All feature columns are numeric.
Skew candidates present: ['number_outpatient', 'number_emergency', 'number_inpatient']


,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,metformin,...,diag_3_grp_Neoplasms (140–239),diag_3_grp_Nervous/Sense (320–389),diag_3_grp_Pregnancy (630–679),diag_3_grp_Respiratory (460–519),diag_3_grp_Skin (680–709),diag_3_grp_Supplementary Factors (V),diag_3_grp_Symptoms/Ill-defined (780–799),diag_3_grp_Unknown,num_diabetes_meds_used,num_med_dose_changes
0,5,1,41,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,15,3,59,0,18,0,0,0,9,0,...,0,0,0,0,0,0,0,0,1,1
2,25,2,11,5,13,2,0,1,6,0,...,0,0,0,0,0,1,0,0,1,0
3,35,2,44,1,16,0,0,0,7,0,...,0,0,0,0,0,0,0,0,1,1
4,45,1,51,0,8,0,0,0,5,0,...,0,0,0,0,0,0,0,0,2,0


In [86]:
raw_data.to_csv("testing.csv", index=False)